In [1]:
import pandas as pd

# Adjusted to use local workspace CSV files
programs = pd.read_csv("programs.csv")
repos = pd.read_csv("repositories.csv")


In [2]:
iac_solutions = ["Terraform", "CDKTF", "Pulumi", "CloudFormation", "Bicep", "Ansible"]
subset = programs[programs["solution"].isin(iac_solutions)]

iac_ext = programs["directory"].str.contains(r"\.tf|cdktf\.json|Pulumi|\.bicep|\.yaml|\.yml", case=False, na=False)
subset = pd.concat([subset, programs[iac_ext]]).drop_duplicates()

merged = subset.merge(repos, left_on="repository", right_on="ID", how="left")
print("Columns:", merged.columns.tolist())
print("Shape:", merged.shape)


Columns: ['ID_x', 'repository', 'directory', 'solution', 'language', 'name_x', 'description_x', 'runtime', 'testing', 'tests', 'ID_y', 'url', 'downloaded', 'name_y', 'description_y', 'licenses', 'redistributable', 'created', 'updated', 'pushed', 'fork', 'forks', 'archive', 'programs']
Shape: (13777, 24)


In [3]:
print(merged.columns.tolist())


['ID_x', 'repository', 'directory', 'solution', 'language', 'name_x', 'description_x', 'runtime', 'testing', 'tests', 'ID_y', 'url', 'downloaded', 'name_y', 'description_y', 'licenses', 'redistributable', 'created', 'updated', 'pushed', 'fork', 'forks', 'archive', 'programs']


In [4]:
cleaned = merged[[
    "ID_y",        # repo ID
    "name_y",      # repo name
    "url",         # repo URL
    "directory",   # IaC folder inside repo
    "solution",
    "language"
]]

print("Cleaned shape:", cleaned.shape)
cleaned.head()


Cleaned shape: (13777, 6)


,ID_y,name_y,url,directory,solution,language
0,386451159,phillipedwards/pulumi-ts-import-error,https://api.github.com/repos/phillipedwards/pu...,386451159,Pulumi,typescript
1,214975348,jforge/iac-samples,https://api.github.com/repos/jforge/iac-samples,214975348/pulumi/quickstart,Pulumi,typescript
2,457090491,eric-sherrill/ahoy-pulumi,https://api.github.com/repos/eric-sherrill/aho...,457090491,Pulumi,typescript
3,453026331,RawkodeAcademy/platform,https://api.github.com/repos/RawkodeAcademy/pl...,453026331/src/index.ts,Pulumi,typescript
4,497668388,iac-factory/gitlab-runner-cdktf,https://api.github.com/repos/iac-factory/gitla...,497668388/cdktf,CDKTF,typescript


In [5]:
import os

# Create download folder
os.makedirs("terraform_repos", exist_ok=True)

# Convert API URL → clone URL
def to_clone_url(api_url):
    return api_url.replace("api.github.com/repos", "github.com") + ".git"

# Pick first 20 repos for testing (you can increase later)
subset = cleaned.head(20)

for idx, row in subset.iterrows():
    clone_url = to_clone_url(row["url"])
    repo_name = row["name_y"].replace("/", "_")  # avoid folder conflicts
    print(f"Cloning {clone_url} into terraform_repos/{repo_name}...")
    os.system(f"git clone --depth 1 {clone_url} terraform_repos/{repo_name}")


Cloning https://github.com/phillipedwards/pulumi-ts-import-error.git into terraform_repos/phillipedwards_pulumi-ts-import-error...
Cloning https://github.com/jforge/iac-samples.git into terraform_repos/jforge_iac-samples...
Cloning https://github.com/eric-sherrill/ahoy-pulumi.git into terraform_repos/eric-sherrill_ahoy-pulumi...


Cloning https://github.com/RawkodeAcademy/platform.git into terraform_repos/RawkodeAcademy_platform...


Cloning https://github.com/iac-factory/gitlab-runner-cdktf.git into terraform_repos/iac-factory_gitlab-runner-cdktf...
Cloning https://github.com/dimitor115/pulumi-instance-refresh-bug-reproduction.git into terraform_repos/dimitor115_pulumi-instance-refresh-bug-reproduction...
Cloning https://github.com/phillipedwards/aws-ts-apigw-lambda.git into terraform_repos/phillipedwards_aws-ts-apigw-lambda...
Cloning https://github.com/NewOrbit/ClientSideEncryption.Demo.git into terraform_repos/NewOrbit_ClientSideEncryption.Demo...
Cloning https://github.com/ahmadalibagheri/cdktf-typescript-aws-iam.git into terraform_repos/ahmadalibagheri_cdktf-typescript-aws-iam...


Cloning https://github.com/andrewlaskey/pulumi-example.git into terraform_repos/andrewlaskey_pulumi-example...
Cloning https://github.com/andrewlaskey/pulumi-example.git into terraform_repos/andrewlaskey_pulumi-example...
Cloning https://github.com/WaffleHacks/infrastructure.git into terraform_repos/WaffleHacks_infrastructure...
Cloning https://github.com/Pocket/firefox-android-home-recommendations.git into terraform_repos/Pocket_firefox-android-home-recommendations...


Cloning https://github.com/nojaf/ronnies.be.git into terraform_repos/nojaf_ronnies.be...
Cloning https://github.com/wjrm500/heroku-clone.git into terraform_repos/wjrm500_heroku-clone...


Cloning https://github.com/rohankumardubey/pulumi.git into terraform_repos/rohankumardubey_pulumi...
Cloning https://github.com/rohankumardubey/pulumi.git into terraform_repos/rohankumardubey_pulumi...
Cloning https://github.com/rohankumardubey/pulumi.git into terraform_repos/rohankumardubey_pulumi...
Cloning https://github.com/rohankumardubey/pulumi.git into terraform_repos/rohankumardubey_pulumi...


Cloning https://github.com/rohankumardubey/pulumi.git into terraform_repos/rohankumardubey_pulumi...


In [6]:
# If python-hcl2 is not installed, run: pip install python-hcl2


In [7]:
import os
import hcl2
import json

def find_iac_files(base_dir):
    iac_files = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".tf") or file.endswith(".json"):
                iac_files.append(os.path.join(root, file))
    return iac_files

def parse_tf_file(file_path):
    try:
        with open(file_path, "r") as f:
            return hcl2.load(f)  # works for .tf
    except Exception:
        try:
            with open(file_path, "r") as f:
                return json.load(f)  # fallback if JSON
        except:
            return None

# Example: parse first few IaC files from cloned repos
sample_dir = "terraform_repos"
files = find_iac_files(sample_dir)
print("Found IaC files:", files[:10])

if files:
    parsed = parse_tf_file(files[0])
    print("Parsed content (sample):", parsed)


Found IaC files: ['terraform_repos\\ahmadalibagheri_cdktf-typescript-aws-iam\\cdktf.json', 'terraform_repos\\ahmadalibagheri_cdktf-typescript-aws-iam\\package-lock.json', 'terraform_repos\\ahmadalibagheri_cdktf-typescript-aws-iam\\package.json', 'terraform_repos\\ahmadalibagheri_cdktf-typescript-aws-iam\\tsconfig.json', 'terraform_repos\\andrewlaskey_pulumi-example\\package-lock.json', 'terraform_repos\\andrewlaskey_pulumi-example\\package.json', 'terraform_repos\\andrewlaskey_pulumi-example\\tsconfig.json', 'terraform_repos\\andrewlaskey_pulumi-example\\child-infra\\package-lock.json', 'terraform_repos\\andrewlaskey_pulumi-example\\child-infra\\package.json', 'terraform_repos\\andrewlaskey_pulumi-example\\child-infra\\tsconfig.json']
Parsed content (sample): {'language': 'typescript', 'app': 'npx ts-node main.ts', 'projectId': 'e100a3f7-9187-4752-b3b5-9f5ef257eccd', 'terraformProviders': ['aws@~> 3.29.0'], 'terraformModules': [], 'context': {'excludeStackIdFromLogicalIds': 'true', 'al

In [8]:
import os
import hcl2
import json

def find_iac_files(base_dir):
    iac_files = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".tf") or file == "cdktf.json" or "Pulumi" in file:
                iac_files.append(os.path.join(root, file))
    return iac_files

def parse_iac_file(file_path):
    try:
        if file_path.endswith(".tf"):
            with open(file_path, "r") as f:
                return hcl2.load(f)
        elif file_path.endswith(".json") or file_path.endswith(".yaml") or file_path.endswith(".yml"):
            with open(file_path, "r") as f:
                return json.load(f)
    except Exception as e:
        return {"error": str(e), "file": file_path}
    return None

# Example: parse IaC files
sample_dir = "terraform_repos"
files = find_iac_files(sample_dir)
print("IaC files found:", files[:10])

parsed_outputs = []
for f in files[:5]:   # try first 5
    parsed_outputs.append((f, parse_iac_file(f)))

for f, content in parsed_outputs:
    print("\nFile:", f)
    print("Parsed content (trimmed):", str(content)[:500])


IaC files found: ['terraform_repos\\ahmadalibagheri_cdktf-typescript-aws-iam\\cdktf.json', 'terraform_repos\\andrewlaskey_pulumi-example\\Pulumi.parent-stack.yaml', 'terraform_repos\\andrewlaskey_pulumi-example\\Pulumi.yaml', 'terraform_repos\\andrewlaskey_pulumi-example\\child-infra\\Pulumi.child-stack.yaml', 'terraform_repos\\andrewlaskey_pulumi-example\\child-infra\\Pulumi.yaml', 'terraform_repos\\dimitor115_pulumi-instance-refresh-bug-reproduction\\Pulumi.dev.yaml', 'terraform_repos\\dimitor115_pulumi-instance-refresh-bug-reproduction\\Pulumi.yaml', 'terraform_repos\\eric-sherrill_ahoy-pulumi\\Pulumi.yaml', 'terraform_repos\\iac-factory_gitlab-runner-cdktf\\cdktf\\cdktf.json', 'terraform_repos\\iac-factory_gitlab-runner-cdktf\\cdktf\\data.tf']

File: terraform_repos\ahmadalibagheri_cdktf-typescript-aws-iam\cdktf.json
Parsed content (trimmed): {'language': 'typescript', 'app': 'npx ts-node main.ts', 'projectId': 'e100a3f7-9187-4752-b3b5-9f5ef257eccd', 'terraformProviders': ['aws@~> 

In [9]:
def extract_features(parsed):
    features = {}
    if not parsed:
        return features

    # Providers
    providers = parsed.get("terraformProviders", [])
    features["num_providers"] = len(providers)
    features["providers"] = providers

    # Modules
    modules = parsed.get("terraformModules", [])
    features["num_modules"] = len(modules)
    features["modules"] = modules

    # Language
    features["language"] = parsed.get("language", "unknown")

    return features

# Run on first 5 parsed configs
for f, content in parsed_outputs:
    if content:
        feats = extract_features(content)
        print("\nFile:", f)
        print("Features:", feats)



File: terraform_repos\ahmadalibagheri_cdktf-typescript-aws-iam\cdktf.json
Features: {'num_providers': 1, 'providers': ['aws@~> 3.29.0'], 'num_modules': 0, 'modules': [], 'language': 'typescript'}

File: terraform_repos\andrewlaskey_pulumi-example\Pulumi.parent-stack.yaml
Features: {'num_providers': 0, 'providers': [], 'num_modules': 0, 'modules': [], 'language': 'unknown'}

File: terraform_repos\andrewlaskey_pulumi-example\Pulumi.yaml
Features: {'num_providers': 0, 'providers': [], 'num_modules': 0, 'modules': [], 'language': 'unknown'}

File: terraform_repos\andrewlaskey_pulumi-example\child-infra\Pulumi.child-stack.yaml
Features: {'num_providers': 0, 'providers': [], 'num_modules': 0, 'modules': [], 'language': 'unknown'}

File: terraform_repos\andrewlaskey_pulumi-example\child-infra\Pulumi.yaml
Features: {'num_providers': 0, 'providers': [], 'num_modules': 0, 'modules': [], 'language': 'unknown'}


In [10]:
import pandas as pd

# Load metadata (local CSVs)
programs = pd.read_csv("programs.csv")
repos = pd.read_csv("repositories.csv")

# 1. Select by known IaC solutions
iac_solutions = ["Terraform", "CDKTF", "Pulumi", "CloudFormation", "Bicep", "Ansible"]
subset = programs[programs["solution"].isin(iac_solutions)]

# 2. Select by file extensions (for safety)
iac_ext = programs["directory"].str.contains(r"\.tf|cdktf\.json|Pulumi|\.bicep|\.yaml|\.yml", case=False, na=False)

# 3. Merge both and deduplicate
subset = pd.concat([subset, programs[iac_ext]]).drop_duplicates()

# 4. Merge with repos metadata
merged = subset.merge(repos, left_on="repository", right_on="ID", how="left")

# 5. Get unique repos
unique_repos = merged.drop_duplicates(subset=["repository"])
print("Unique IaC repos:", unique_repos.shape[0])

# 6. If repos < 5000, just take all
if unique_repos.shape[0] >= 5000:
    sampled_repos = unique_repos.sample(n=5000, random_state=42)
else:
    sampled_repos = unique_repos  # take all available
    print(f"Only {unique_repos.shape[0]} repos available, taking all.")

# Save list
sampled_repos[["repository", "name_y", "url", "directory"]].to_csv("iac_sampled.csv", index=False)
print("Saved sampled repo list to iac_sampled.csv")


Unique IaC repos: 3885
Only 3885 repos available, taking all.
Saved sampled repo list to iac_sampled.csv


In [11]:
print(unique_repos["solution"].value_counts())


solution
Pulumi    3366
CDKTF      519
Name: count, dtype: int64


In [12]:
import os, subprocess, pandas as pd

# Load repo list
repos = pd.read_csv("iac_sampled.csv")

# Create folder for clones
os.makedirs("iac_repos", exist_ok=True)

# Clone first 50 repos as a test (safe for Review-2)
for i, row in repos.head(50).iterrows():
    repo_url = row["url"].replace("api.github.com/repos", "github.com")  # fix API → git URL
    repo_name = row["name_y"].replace("/", "_")
    target_dir = f"iac_repos/{repo_name}"

    if not os.path.exists(target_dir):
        try:
            print(f"Cloning {repo_url}...")
            subprocess.run(["git", "clone", repo_url, target_dir], check=True)
        except Exception as e:
            print(f"Failed {repo_url}: {e}")


Cloning https://github.com/RawkodeAcademy/platform...


Failed https://github.com/RawkodeAcademy/platform: Command '['git', 'clone', 'https://github.com/RawkodeAcademy/platform', 'iac_repos/RawkodeAcademy_platform']' returned non-zero exit status 128.
Cloning https://github.com/JIO93/Programming-With-Python-For-DevOps-Engineers-Bootcamp...


Failed https://github.com/JIO93/Programming-With-Python-For-DevOps-Engineers-Bootcamp: Command '['git', 'clone', 'https://github.com/JIO93/Programming-With-Python-For-DevOps-Engineers-Bootcamp', 'iac_repos/JIO93_Programming-With-Python-For-DevOps-Engineers-Bootcamp']' returned non-zero exit status 128.
Cloning https://github.com/nyamada43/demoPulumi...


Failed https://github.com/nyamada43/demoPulumi: Command '['git', 'clone', 'https://github.com/nyamada43/demoPulumi', 'iac_repos/nyamada43_demoPulumi']' returned non-zero exit status 128.


In [13]:
import os, subprocess, pandas as pd, time

# Load repo list (already filtered IaC repos)
repos = pd.read_csv("iac_sampled.csv")

# Create folder to store repos
os.makedirs("iac_repos", exist_ok=True)

# Define the clone function again
def clone_repo(url, name):
    repo_url = url.replace("api.github.com/repos", "github.com")
    target_dir = f"iac_repos/{name.replace('/', '_')}"
    if os.path.exists(target_dir):
        print(f"Already exists: {name}")
        return

    try:
        print(f"Cloning {repo_url}...")
        subprocess.run(
            ["git", "clone", "--depth", "1", repo_url, target_dir],
            check=True
        )
    except Exception as e:
        print(f"Failed {repo_url}: {e}")

# Batch clone only 1000 repos (safe for Review 2)
for i, row in repos.iloc[:1000].iterrows():
    clone_repo(row["url"], row["name_y"])

    if i % 200 == 0 and i > 0:
        print("Pausing to avoid GitHub block...")
        time.sleep(60)  # 1 min pause every 200 repos


Already exists: phillipedwards/pulumi-ts-import-error
Already exists: jforge/iac-samples
Already exists: eric-sherrill/ahoy-pulumi
Cloning https://github.com/RawkodeAcademy/platform...


Failed https://github.com/RawkodeAcademy/platform: Command '['git', 'clone', '--depth', '1', 'https://github.com/RawkodeAcademy/platform', 'iac_repos/RawkodeAcademy_platform']' returned non-zero exit status 128.
Already exists: iac-factory/gitlab-runner-cdktf
Already exists: dimitor115/pulumi-instance-refresh-bug-reproduction
Already exists: phillipedwards/aws-ts-apigw-lambda
Already exists: NewOrbit/ClientSideEncryption.Demo
Already exists: ahmadalibagheri/cdktf-typescript-aws-iam
Already exists: andrewlaskey/pulumi-example
Already exists: WaffleHacks/infrastructure
Already exists: Pocket/firefox-android-home-recommendations
Already exists: nojaf/ronnies.be
Already exists: wjrm500/heroku-clone
Already exists: rohankumardubey/pulumi
Already exists: alek29c/tf
Already exists: AdminTurnedDevOps/Live-Sessions-And-Conferences
Already exists: andrekiba/FantasticBike
Already exists: PHONGLEX/fastapi_pulumi_practice
Already exists: cyrano5614/pulumi-gcp-network
Already exists: hossambarakat/p

Failed https://github.com/JIO93/Programming-With-Python-For-DevOps-Engineers-Bootcamp: Command '['git', 'clone', '--depth', '1', 'https://github.com/JIO93/Programming-With-Python-For-DevOps-Engineers-Bootcamp', 'iac_repos/JIO93_Programming-With-Python-For-DevOps-Engineers-Bootcamp']' returned non-zero exit status 128.
Already exists: matrixbegins/uni-demo-deploy
Already exists: Shyam-Chen/Micro-Fullstack
Already exists: jaxxstorm/pulumi-automationapi-workshop
Already exists: cloudspeak/pulumi-lambda-efs
Already exists: aeons/aeons.dk
Already exists: mitodl/ol-infrastructure
Already exists: just-run-my/app-azure-edition
Cloning https://github.com/nyamada43/demoPulumi...


Failed https://github.com/nyamada43/demoPulumi: Command '['git', 'clone', '--depth', '1', 'https://github.com/nyamada43/demoPulumi', 'iac_repos/nyamada43_demoPulumi']' returned non-zero exit status 128.
Already exists: skorfmann/cdktf-provider-aws
Already exists: pulumi/pulumi-azure-quickstart-compute
Already exists: joeduffy/pulumi-awsvpc
Already exists: pulumi/pulumi-mailgun
Already exists: melihyazgan/IaC_pipeline
Already exists: EwanNoble/AppCatalog
Already exists: ikovac/pulumi-workshop-1
Already exists: nmora53/async-power-dialer-stone-poc
Already exists: artcoded-net/vendure-pulumi-gcp
Already exists: hmix/try-terraform-cdk-python
Already exists: jessicaUP/CloudVisitCount
Cloning https://github.com/strongishllama/cdktf-constructs...


Failed https://github.com/strongishllama/cdktf-constructs: Command '['git', 'clone', '--depth', '1', 'https://github.com/strongishllama/cdktf-constructs', 'iac_repos/strongishllama_cdktf-constructs']' returned non-zero exit status 128.
Already exists: pulumi/pulumi-mysql
Already exists: brightsole/solosis
Already exists: L0ndra/pulumi-do-ddos
Already exists: neilkuan/pulumi-cdk-example
Cloning https://github.com/somandr/ISaC...


...

Cloning https://github.com/khaiyik2612/PulumiCICS...


Failed https://github.com/khaiyik2612/PulumiCICS: Command '['git', 'clone', '--depth', '1', 'https://github.com/khaiyik2612/PulumiCICS', 'iac_repos/khaiyik2612_PulumiCICS']' returned non-zero exit status 128.
Cloning https://github.com/milanpollock/dark-rush-photography...


Cloning https://github.com/Kapernikov/skaffold-helm-tutorial...


Cloning https://github.com/Pocket/recommendation-api...


KeyboardInterrupt: 

In [26]:
import shutil, os

base = "iac_repos"
remove_dirs = ["node_modules", ".git", "vendor", "__pycache__"]

for repo in os.listdir(base):
    for root, dirs, files in os.walk(os.path.join(base, repo)):
        for d in dirs:
            if d in remove_dirs:
                path = os.path.join(root, d)
                try:
                    shutil.rmtree(path)
                    print("Removed", path)
                except Exception as e:
                    pass


Removed iac_repos/gobengo_activitypub.com/.git
Removed iac_repos/rgl_dotnet-pulumi-libvirt-rke-example/.git
Removed iac_repos/sinanbekar_turborepo-terraform-cdk-aws-vercel/.git
Removed iac_repos/paiml_python_devops_book/.git
Removed iac_repos/paiml_python_devops_book/src/chap13-Serverless/google-python-simple-http-endpoint/node_modules
Removed iac_repos/phillipedwards_pulumi-ts-import-error/.git
Removed iac_repos/cjh-cloud_esp32_pwa_starter/.git
Removed iac_repos/Addono_guide-to-pulumi-demo--common-infra/.git
Removed iac_repos/JessicaRudd_mlplatform-infra/.git
Removed iac_repos/pierskarsenbarg_aws-iam-demo/.git
Removed iac_repos/Ishanamgai_NFT-DApp/.git
Removed iac_repos/Ishanamgai_NFT-DApp/webapp/src/modules/vendor
Removed iac_repos/MarcinGadz_iaac-comparision/.git
Removed iac_repos/w9n_pulumi-kind/.git
Removed iac_repos/sthuck_duckduckstay/.git
Removed iac_repos/hashicorp_cdktf-provider-random/.git
Removed iac_repos/DanielMSchmidt_cdktf-multi-stack-tfe/.git
Removed iac_repos/anonymou

In [27]:
import os, shutil, random

repos = os.listdir("iac_repos")
print("Total repos available:", len(repos))

# Pick all available repos (or up to 1000 if you have more)
sample_size = min(1000, len(repos))
sample = random.sample(repos, sample_size)

subset_dir = "iac_subset"
os.makedirs(subset_dir, exist_ok=True)

# Directories we want to ignore (junk)
ignore_dirs = ["node_modules", ".git", "venv", "__pycache__", "dist", "build"]

def ignore_patterns(_, names):
    return [n for n in names if n in ignore_dirs]

for r in sample:
    src = os.path.join("iac_repos", r)
    dst = os.path.join(subset_dir, r)
    if not os.path.exists(dst):
        try:
            shutil.copytree(src, dst, ignore=ignore_patterns)
            print("Copied:", r)
        except Exception as e:
            print("Failed:", r, e)



Total repos available: 936
Copied: sgrade_pulumi
Copied: Jmen_hello-pulumi
Copied: elbetasal_app-playground
Copied: Ishanamgai_NFT-DApp
Copied: justinvp_secret-outputs-repro
Copied: pulumi_pulumi
Copied: vavsab_pulumi-aiven-import-issue
Copied: pulumi_pulumi-confluentcloud
Copied: rohankumardubey_kubernetes-operator
Copied: lindydonna_velocity-examples
Copied: donbing_Pairy
Copied: syahiaoui_pulumi-deploy-clourun
Copied: bloominlabs_baseplate-go
Copied: martinothamar_EasyAzureWebApp
Copied: kubarozycki_infra-as-a-code-exercise
Copied: squaremo_pulumi-ociregistry-provider
Copied: pulumi_pulumi-auth0
Copied: kingster307_localstack_ws_bugs
Copied: andrekiba_CrazyBike
Copied: xiehan_cdktf-integration-alexa-example
Copied: JulesP96_Sample-Pulumi
Copied: iac-factory_gitlab-runner-cdktf
Copied: uma-arai_iac-story-code
Copied: RahulMR42_pulumi-python-oci-instances
Copied: RafaelFigueiredo_pulumi-digitalocean-k8s-linkerd
Copied: marcaurele_pulumi-cloud-demo
Copied: lakindu2002_pulumi-micro-stac

In [28]:
import os, pandas as pd

features = []

for root, _, files in os.walk("iac_subset"):
    for f in files:
        if f.endswith((".tf", ".json", ".yml", ".yaml", ".bicep")):
            path = os.path.join(root, f)

            try:
                size = os.path.getsize(path)
                if size > 1_000_000:  # Skip >1MB files
                    continue

                # quick check if file is readable
                valid = False
                try:
                    with open(path, "r", errors="ignore") as raw:
                        snippet = raw.read(300).lower()
                        valid = True if snippet else False
                except:
                    snippet = ""

                features.append({
                    "repo": os.path.basename(root),
                    "file": f,
                    "type": f.split(".")[-1],
                    "size_bytes": size,
                    "lines": sum(1 for _ in open(path, "r", errors="ignore")),
                    "parsed": valid,
                    "raw_text_snippet": snippet
                })
            except Exception:
                pass

df = pd.DataFrame(features)
df.to_csv("iac_features_raw.csv", index=False)
print("✅ Saved parsed base dataset to iac_features_raw.csv")
print("Total IaC files parsed:", len(df))
print(df.head())


✅ Saved parsed base dataset to iac_features_raw.csv
Total IaC files parsed: 35816
                      repo               file  type  size_bytes  lines  \
0  gobengo_activitypub.com   Pulumi.prod.yaml  yaml         281      7   
1  gobengo_activitypub.com  package-lock.json  json      118347   2557   
2  gobengo_activitypub.com        Pulumi.yaml  yaml          97      3   
3  gobengo_activitypub.com       package.json  json         343     14   
4  gobengo_activitypub.com    Pulumi.dev.yaml  yaml         321      7   

   parsed                                   raw_text_snippet  
0    True  config:\n  activitypub.com-cloud:apex-dns-stac...  
1    True  {\n    "name": "activitypub.com-cloud",\n    "...  
2    True  name: activitypub.com-cloud\nruntime: nodejs\n...  
3    True  {\n    "name": "activitypub.com-cloud",\n    "...  
4    True  config:\n  activitypub.com-cloud:apex-dns-stac...  


In [29]:
import os
import pandas as pd
from tqdm import tqdm

# Define IaC file extensions
iac_exts = (".tf", ".json", ".yaml", ".yml", ".bicep")
base_dir = "iac_repos"

records = []

# Walk all repos
for repo in tqdm(os.listdir(base_dir), desc="Repos"):
    repo_path = os.path.join(base_dir, repo)
    for root, dirs, files in os.walk(repo_path):
        for f in files:
            if f.endswith(iac_exts) or f.lower() in ["cdktf.json", "pulumi.yaml"]:
                file_path = os.path.join(root, f)
                try:
                    with open(file_path, "r", encoding="utf-8", errors="ignore") as fh:
                        content = fh.read()
                    records.append({
                        "repo": repo,
                        "file": os.path.relpath(file_path, repo_path),
                        "type": f.split(".")[-1].lower(),
                        "size_bytes": os.path.getsize(file_path),
                        "lines": content.count("\n") + 1,
                        "full_content": content
                    })
                except Exception as e:
                    print(f"⚠️ Error reading {file_path}: {e}")

# Convert to DataFrame
df = pd.DataFrame(records)
print("Total IaC files parsed:", df.shape[0])

# Save as Parquet (compressed, efficient)
df.to_parquet("iac_files_full.parquet", engine="pyarrow", compression="snappy")

print("✅ Saved full IaC dataset to iac_files_full.parquet")


Repos:  60%|█████▉    | 561/936 [00:13<00:08, 42.91it/s]

⚠️ Error reading iac_repos/nullify005_HomeAssistant/app/config/service_account.json: [Errno 2] No such file or directory: 'iac_repos/nullify005_HomeAssistant/app/config/service_account.json'
⚠️ Error reading iac_repos/nullify005_HomeAssistant/app/config/secrets.yaml: [Errno 2] No such file or directory: 'iac_repos/nullify005_HomeAssistant/app/config/secrets.yaml'


Repos: 100%|██████████| 936/936 [00:27<00:00, 34.03it/s]


Total IaC files parsed: 35278
✅ Saved full IaC dataset to iac_files_full.parquet


In [ ]:
# Colab-specific drive mount removed for local execution


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Skipping Google Drive backup when running locally.
print('Skipping Drive backup; running locally.')


✅ Backup saved to Google Drive at /content/drive/MyDrive/iac_files_full.parquet


In [ ]:
# If checkov is not installed, run: pip install checkov


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.4/361.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.6/120.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2

In [1]:
import pandas as pd
df = pd.read_parquet("iac_files_full.parquet")
print(df.shape)


(35278, 6)


In [2]:
import os

subset_dir = "iac_subset"
iac_files = []

for root, _, files in os.walk(subset_dir):
    for f in files:
        if f.endswith((".tf", ".yaml", ".yml", ".json", ".bicep")):
            path = os.path.join(root, f)
            with open(path, "r", errors="ignore") as fh:
                head = fh.read(200).lower()
                # Look for IaC markers
                if any(keyword in head for keyword in ["resource", "provider", "apiVersion", "kind", "Resources:"]):
                    iac_files.append(path)

print("Found IaC-like files:", len(iac_files))
print("Examples:")
for f in iac_files[:5]:
    print(" -", f)


Found IaC-like files: 11895
Examples:
 - iac_subset/paiml_python_devops_book/src/chap10-InfraAsCode/terraform/main.tf
 - iac_subset/paiml_python_devops_book/src/chap10-InfraAsCode/terraform/modules/route53/main.tf
 - iac_subset/paiml_python_devops_book/src/chap10-InfraAsCode/terraform/modules/s3/main.tf
 - iac_subset/paiml_python_devops_book/src/chap10-InfraAsCode/terraform/modules/cloudfront/main.tf
 - iac_subset/paiml_python_devops_book/src/chap10-InfraAsCode/terraform/modules/acm/main.tf


In [6]:
import subprocess, json

# Use explicit Checkov command path found on this Windows system
CHECKOV_CMD = r"C:\Users\chava\AppData\Roaming\Python\Python311\Scripts\checkov.cmd"

def run_checkov(file_path, timeout=30):
    """Run Checkov on one file and return parsed JSON result."""
    try:
        cmd = [CHECKOV_CMD, "-f", file_path, "-o", "json"]
        out = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
        if out.returncode == 0 and out.stdout.strip():
            return json.loads(out.stdout)
        else:
            # Print stderr for clearer diagnostics
            stderr = out.stderr.strip() if out and out.stderr else ''
            print(f"⚠️ Checkov returned no results or error on {file_path}: rc={out.returncode}, stderr={stderr}")
    except subprocess.TimeoutExpired:
        print(f"⏳ Timeout on {file_path}")
    except Exception as e:
        print(f"⚠️ Error running Checkov on {file_path}: {e}")
    return None


In [ ]:
test_file = "iac_subset/paiml_python_devops_book/src/chap10-InfraAsCode/terraform/main.tf"
result = run_checkov(test_file, timeout=30)

if result and "results" in result:
    failed = result["results"].get("failed_checks", [])
    print("🔍 File:", test_file)
    print("   Failed checks:", len(failed))
    if failed:
        print("   Example issue:", failed[0].get("check_id"), "-", failed[0].get("check_name"))
else:
    print("❌ No result")


🔍 File: iac_subset/paiml_python_devops_book/src/chap10-InfraAsCode/terraform/main.tf
   Failed checks: 0


: 

In [ ]:
import pandas as pd
df = pd.DataFrame(results)

# Save initial Checkov results locally
save_path = "checkov_results_local.csv"
df.to_csv(save_path, index=False)

print(f"✅ Initial Checkov results saved to {save_path}")
print("Shape:", df.shape)
print(df.head())


✅ Initial Checkov results saved to /content/drive/MyDrive/checkov_results_partial.csv
Shape: (2, 6)
                                                file     check_id  \
0  iac_subset/paiml_python_devops_book/src/chap10...   CKV_AWS_70   
1  iac_subset/paiml_python_devops_book/src/chap10...  CKV_AWS_310   

                                          check_name severity  \
0  Ensure S3 bucket does not allow an action with...     HIGH   
1  Ensure CloudFront distributions should have or...   MEDIUM   

                              resource  \
0                aws_s3_bucket.example   
1  aws_cloudfront_distribution.example   

                                           guideline  
0  https://docs.bridgecrew.io/docs/s3_1-acl-read-...  
1  https://docs.bridgecrew.io/docs/cloudfront_1-e...  


In [ ]:
# Running locally: Google Colab drive mount is not available in this environment.
# If you run on Colab and need to remount, call: drive.mount('/content/drive', force_remount=True)
print('Running locally; not attempting to mount /content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
# Load local parquet and checkov CSV if available
try:
    df = pd.read_parquet("iac_files_full.parquet")
except Exception as e:
    print('Failed to load iac_files_full.parquet:', e)
    df = pd.DataFrame()
try:
    scanned = pd.read_csv("checkov_results_local.csv")
except Exception:
    scanned = pd.DataFrame()
print('IaC rows:', getattr(df, 'shape', None), 'Checkov rows:', getattr(scanned, 'shape', None))


(35278, 6) (2, 6)


In [ ]:
# Colab-specific drive mount removed for local execution

# Load parsed IaC dataset from local path if available
import pandas as pd
try:
    iac_files = pd.read_parquet("iac_files_full.parquet")
    print("IaC dataset loaded:", iac_files.shape)
except FileNotFoundError:
    print("Local iac_files_full.parquet not found. Skipping this step.")

# Load Checkov results (if exists)
try:
    checkov_results = pd.read_csv("checkov_results_partial.csv")
    print("Checkov results loaded:", checkov_results.shape)
except FileNotFoundError:
    checkov_results = pd.DataFrame(columns=["file","check_id","check_name","severity","resource","guideline"])
    print("No existing Checkov results, starting fresh")

# Show samples (if available)
try:
    display(iac_files.head())
except Exception:
    pass
try:
    display(checkov_results.head())
except Exception:
    pass


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
IaC dataset loaded: (35278, 6)
Checkov results loaded: (2, 6)


,repo,file,type,size_bytes,lines,full_content
0,gobengo_activitypub.com,Pulumi.prod.yaml,yaml,281,8,config:\n activitypub.com-cloud:apex-dns-stac...
1,gobengo_activitypub.com,package-lock.json,json,118347,2558,"{\n ""name"": ""activitypub.com-cloud"",\n ""..."
2,gobengo_activitypub.com,Pulumi.yaml,yaml,97,4,name: activitypub.com-cloud\nruntime: nodejs\n...
3,gobengo_activitypub.com,package.json,json,343,15,"{\n ""name"": ""activitypub.com-cloud"",\n ""..."
4,gobengo_activitypub.com,Pulumi.dev.yaml,yaml,321,8,config:\n activitypub.com-cloud:apex-dns-stac...


,file,check_id,check_name,severity,resource,guideline
0,iac_subset/paiml_python_devops_book/src/chap10...,CKV_AWS_70,Ensure S3 bucket does not allow an action with...,HIGH,aws_s3_bucket.example,https://docs.bridgecrew.io/docs/s3_1-acl-read-...
1,iac_subset/paiml_python_devops_book/src/chap10...,CKV_AWS_310,Ensure CloudFront distributions should have or...,MEDIUM,aws_cloudfront_distribution.example,https://docs.bridgecrew.io/docs/cloudfront_1-e...


In [3]:
# Aggregate Checkov findings per file and merge with IaC dataset
import pandas as pd

# Ensure datasets exist
try:
    iac_files = pd.read_parquet("iac_files_full.parquet")
    print('Loaded iac_files_full.parquet:', iac_files.shape)
except Exception as e:
    print('iac_files_full.parquet not available locally:', e)
    iac_files = pd.DataFrame()

try:
    checkov_results = pd.read_csv("checkov_results_local.csv")
    print('Loaded checkov_results_local.csv:', checkov_results.shape)
except Exception as e:
    print('checkov_results_local.csv not available:', e)
    checkov_results = pd.DataFrame(columns=["file","check_id","check_name","severity","resource","guideline"]) 

# If Checkov results are present, aggregate per file
if not checkov_results.empty:
    # Group check_ids and severities per file
    agg = checkov_results.groupby('file').agg({
        'check_id': lambda ids: list(pd.unique([i for i in ids if pd.notna(i)])),
        'severity': lambda s: list(pd.unique([i for i in s if pd.notna(i)])),
        'check_name': lambda names: list(pd.unique([n for n in names if pd.notna(n)])),
        'resource': lambda r: list(pd.unique([x for x in r if pd.notna(x)])),
    }).reset_index()
    agg = agg.rename(columns={'check_id': 'check_ids', 'severity': 'severities', 'check_name': 'check_names', 'resource': 'resources'})
else:
    agg = pd.DataFrame(columns=['file','check_ids','severities','check_names','resources'])

# Merge with IaC files (if available). The 'file' column in checkov_results likely contains full paths; attempt to match on suffix.
if not iac_files.empty and not agg.empty:
    # Normalize file paths: make both use forward slashes and compare suffixes
    iac_files['normalized_path'] = iac_files['file'].str.replace('\\\\', '/', regex=True)
    agg['normalized_file'] = agg['file'].str.replace('\\\\', '/', regex=True)

    # Attempt direct join first
    merged = iac_files.merge(agg, left_on='file', right_on='file', how='left')

    # If no matches, try matching by suffix (endswith)
    if merged['check_ids'].isna().all():
        print('Direct file join yielded no matches; attempting suffix match')
        # Build a mapping from basename or suffix to agg row
        agg_indexed = agg.set_index('file')
        def find_checks_for_path(path):
            # exact
            if path in agg_indexed.index:
                return agg_indexed.loc[path].to_dict()
            # suffix match
            for k in agg_indexed.index:
                if path.endswith(k):
                    return agg_indexed.loc[k].to_dict()
            return {'check_ids': [], 'severities': [], 'check_names': [], 'resources': []}

        merged_rows = []
        for _, row in iac_files.iterrows():
            path = row['file']
            checks = find_checks_for_path(path)
            new = row.to_dict()
            new.update(checks)
            merged_rows.append(new)
        merged = pd.DataFrame(merged_rows)
else:
    # If IaC files not available, just save aggregated Checkov results
    if not agg.empty:
        merged = agg.copy()
    else:
        merged = pd.DataFrame()

# Save merged output
out_path = 'iac_with_checkov.csv'
try:
    merged.to_csv(out_path, index=False)
    print('Saved merged IaC + Checkov to', out_path)
    print('Merged shape:', getattr(merged, 'shape', None))
except Exception as e:
    print('Failed to save merged CSV:', e)

# Show sample rows
try:
    display(merged.head())
except Exception:
    print('Merged preview not available')


iac_files_full.parquet not available locally: [Errno 2] No such file or directory: 'iac_files_full.parquet'
Loaded checkov_results_local.csv: (206, 2)


KeyError: 'file'

In [2]:
# Diagnostic: inspect checkov results CSV structure
import pandas as pd
import os

csv_path = 'checkov_results_local.csv'
if os.path.exists(csv_path):
    df_check = pd.read_csv(csv_path)
    print('Shape:', df_check.shape)
    print('\nColumns:')
    print(df_check.columns.tolist())
    print('\nDtypes:')
    print(df_check.dtypes)
    print('\nSample rows:')
    display(df_check.head(10))
else:
    print('No checkov_results_local.csv found')


Shape: (103, 2)

Columns:
['target', 'error']

Dtypes:
target    object
error     object
dtype: object

Sample rows:


,target,error
0,d:\CloudGuardAI\iac_subset\DanManson_gcp.iamCu...,[WinError 193] %1 is not a valid Win32 applica...
1,d:\CloudGuardAI\iac_subset\ChanghoonHyun_cdktf...,[WinError 193] %1 is not a valid Win32 applica...
2,d:\CloudGuardAI\iac_subset\Bharathkumarraju_gc...,[WinError 193] %1 is not a valid Win32 applica...
3,d:\CloudGuardAI\iac_subset\AlfredoPardo-zz_pyt...,[WinError 193] %1 is not a valid Win32 applica...
4,d:\CloudGuardAI\iac_subset\DanielMSchmidt_cdkt...,[WinError 193] %1 is not a valid Win32 applica...
5,d:\CloudGuardAI\iac_subset\DanielMSchmidt_cdkt...,[WinError 193] %1 is not a valid Win32 applica...
6,d:\CloudGuardAI\iac_subset\DanielMSchmidt_cdkt...,[WinError 193] %1 is not a valid Win32 applica...
7,d:\CloudGuardAI\iac_subset\DanielMSchmidt_terr...,[WinError 193] %1 is not a valid Win32 applica...
8,d:\CloudGuardAI\iac_subset\JakeGinnivan_pulumi...,[WinError 193] %1 is not a valid Win32 applica...
9,d:\CloudGuardAI\iac_subset\DevOpsRandD_iac_cdk...,[WinError 193] %1 is not a valid Win32 applica...


In [10]:
import os, subprocess, json, pandas as pd
from tqdm import tqdm

# Paths
subset_dir = "iac_subset"
results_path = "/content/drive/MyDrive/checkov_results_partial.csv"

# Load existing results
try:
    checkov_results = pd.read_csv(results_path)
    print("Loaded existing results:", checkov_results.shape)
except FileNotFoundError:
    checkov_results = pd.DataFrame(columns=["file","check_id","check_name","severity","resource","guideline"])
    print("No existing results, starting fresh")

# Collect IaC files
iac_files = []
for root, _, files in os.walk(subset_dir):
    for f in files:
        if f.endswith((".tf", ".json", ".yml", ".yaml", ".bicep")):
            iac_files.append(os.path.join(root, f))

print("Total IaC files:", len(iac_files))

# Remove files already scanned
done_files = set(checkov_results["file"].tolist())
pending_files = [f for f in iac_files if f not in done_files]
print("Already scanned:", len(done_files))
print("Pending:", len(pending_files))

def run_checkov(file_path, timeout=30):
    """Run checkov on one file with a timeout."""
    try:
        cmd = ["checkov", "-f", file_path, "-o", "json"]
        out = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
        if out.returncode == 0 and out.stdout.strip():
            return json.loads(out.stdout)
    except subprocess.TimeoutExpired:
        print(f"⏳ Timeout on {file_path}")
    except Exception as e:
        print(f"⚠️ Error on {file_path}: {e}")
    return None

# Batch scanning (small chunks to be safe)
batch_size = 20
for i in tqdm(range(0, len(pending_files), batch_size), desc="Batches"):
    batch = pending_files[i:i+batch_size]
    new_results = []

    for file_path in batch:
        result = run_checkov(file_path, timeout=30)
        if result and "results" in result:
            for failed_check in result["results"].get("failed_checks", []):
                new_results.append({
                    "file": file_path,
                    "check_id": failed_check.get("check_id"),
                    "check_name": failed_check.get("check_name"),
                    "severity": failed_check.get("severity"),
                    "resource": failed_check.get("resource"),
                    "guideline": failed_check.get("guideline")
                })

    # Append new batch results
    if new_results:
        checkov_results = pd.concat([checkov_results, pd.DataFrame(new_results)], ignore_index=True)
        checkov_results.to_csv(results_path, index=False)
        print(f"💾 Saved progress: {checkov_results.shape}")

print("✅ Scanning finished")


Loaded existing results: (2, 6)
Total IaC files: 35966
Already scanned: 2
Pending: 35964


Batches:   0%|          | 0/1799 [00:33<?, ?it/s]


KeyboardInterrupt: 

In [11]:
import os
for root, _, files in os.walk("iac_subset"):
    for f in files[:10]:  # show 10 examples
        print(os.path.join(root, f))


Streaming output truncated to the last 5000 lines.
iac_subset/ServiTrace_ReplicationPackage/applications/pulumi-examples/aws-go-console-slack-notification/main.go
iac_subset/ServiTrace_ReplicationPackage/applications/pulumi-examples/aws-go-console-slack-notification/go.sum
iac_subset/ServiTrace_ReplicationPackage/applications/pulumi-examples/aws-go-console-slack-notification/README.md
iac_subset/ServiTrace_ReplicationPackage/applications/pulumi-examples/aws-go-console-slack-notification/handler/handler.go
iac_subset/ServiTrace_ReplicationPackage/applications/pulumi-examples/aws-ts-eks-migrate-nodegroups/echoserver.ts
iac_subset/ServiTrace_ReplicationPackage/applications/pulumi-examples/aws-ts-eks-migrate-nodegroups/utils.ts
iac_subset/ServiTrace_ReplicationPackage/applications/pulumi-examples/aws-ts-eks-migrate-nodegroups/iam.ts
iac_subset/ServiTrace_ReplicationPackage/applications/pulumi-examples/aws-ts-eks-migrate-nodegroups/Pulumi.yaml
iac_subset/ServiTrace_ReplicationPackage/applic

In [12]:
import pandas as pd

df = pd.read_parquet("/content/drive/MyDrive/iac_files_full.parquet")
print("IaC dataset restored:", df.shape)


IaC dataset restored: (35278, 6)


In [13]:
import os

subset_dir = "iac_temp"
os.makedirs(subset_dir, exist_ok=True)

# Save 100 sample files for Checkov testing
for i, row in df.head(100).iterrows():
    fpath = os.path.join(subset_dir, f"{row['repo']}_{i}.{row['type']}")
    with open(fpath, "w") as f:
        f.write(row["full_content"])


In [14]:
import subprocess, json

def run_checkov(file_path):
    try:
        cmd = ["checkov", "-f", file_path, "-o", "json"]
        out = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        if out.returncode == 0 and out.stdout.strip():
            return json.loads(out.stdout)
    except Exception as e:
        print("⚠️", e)
    return None


In [16]:
import os

subset_dir = "iac_subset"   # or "iac_repos" if you kept all repos
iac_files = []

for root, _, files in os.walk(subset_dir):
    for f in files:
        if f.endswith((".tf", ".json", ".yml", ".yaml", ".bicep")):
            iac_files.append(os.path.join(root, f))

print("Found IaC files:", len(iac_files))
print("Examples:", iac_files[:5])


Found IaC files: 35966
Examples: ['iac_subset/gobengo_activitypub.com/Pulumi.prod.yaml', 'iac_subset/gobengo_activitypub.com/package-lock.json', 'iac_subset/gobengo_activitypub.com/Pulumi.yaml', 'iac_subset/gobengo_activitypub.com/package.json', 'iac_subset/gobengo_activitypub.com/Pulumi.dev.yaml']


In [17]:
import subprocess, json

def run_checkov(file_path, timeout=60):
    """Run Checkov on a file and return JSON results if available."""
    try:
        cmd = ["checkov", "-f", file_path, "-o", "json"]
        out = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
        if out.returncode == 0 and out.stdout.strip():
            return json.loads(out.stdout)
    except subprocess.TimeoutExpired:
        print(f"⏳ Timeout on {file_path}")
    except Exception as e:
        print(f"⚠️ Error on {file_path}: {e}")
    return None


In [19]:
import pandas as pd, os, subprocess, json
from tqdm import tqdm

# Load IaC files
subset_dir = "iac_subset"
iac_files = []
for root, _, files in os.walk(subset_dir):
    for f in files:
        if f.endswith((".tf", ".json", ".yml", ".yaml", ".bicep")):
            iac_files.append(os.path.join(root, f))

print("Total IaC files:", len(iac_files))

# Load already scanned
if os.path.exists("checkov_results_partial.csv"):
    scanned = pd.read_csv("checkov_results_partial.csv")["file"].unique().tolist()
else:
    scanned = []

pending_files = [f for f in iac_files if f not in scanned]
print("Pending files:", len(pending_files))


def run_checkov(file_path, timeout=60):
    try:
        cmd = ["checkov", "-f", file_path, "-o", "json"]
        out = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
        if out.returncode == 0 and out.stdout.strip():
            return json.loads(out.stdout)
    except subprocess.TimeoutExpired:
        print(f"⏳ Timeout on {file_path}")
    except Exception as e:
        print(f"⚠️ Error on {file_path}: {e}")
    return None


# Batch size small for testing
batch_size = 100
results = []

for i in tqdm(range(0, len(pending_files), batch_size), desc="Batches"):
    batch = pending_files[i:i+batch_size]

    for file_path in batch:
        result = run_checkov(file_path, timeout=30)
        if result and "results" in result:
            for failed_check in result["results"].get("failed_checks", []):
                results.append({
                    "file": file_path,
                    "check_id": failed_check.get("check_id"),
                    "check_name": failed_check.get("check_name"),
                    "severity": failed_check.get("severity"),
                    "resource": failed_check.get("resource"),
                    "guideline": failed_check.get("guideline")
                })

    # Append results to file (instead of overwrite)
    df = pd.DataFrame(results)
    if os.path.exists("checkov_results_partial.csv"):
        df.to_csv("checkov_results_partial.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("checkov_results_partial.csv", index=False)

    results = []  # clear memory
    break  # <-- REMOVE later; for now, process only 1 batch to test


Total IaC files: 35966
Pending files: 35966


Batches:   0%|          | 0/360 [01:44<?, ?it/s]


KeyboardInterrupt: 

In [20]:
import os, subprocess, json, pandas as pd
from tqdm import tqdm

subset_dir = "iac_subset"

# Collect IaC-like files
iac_files = []
for root, _, files in os.walk(subset_dir):
    for f in files:
        if f.endswith((".tf", ".json", ".yml", ".yaml", ".bicep")):
            iac_files.append(os.path.join(root, f))

print("Total IaC files:", len(iac_files))

# Load already scanned
if os.path.exists("checkov_results_partial.csv"):
    scanned = pd.read_csv("checkov_results_partial.csv")["file"].unique().tolist()
else:
    scanned = []

pending_files = [f for f in iac_files if f not in scanned]
print("Pending files:", len(pending_files))


def run_checkov(file_path, timeout=30):
    """Run Checkov on one file and return parsed results."""
    try:
        cmd = ["checkov", "-f", file_path, "-o", "json"]
        out = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
        if out.returncode == 0 and out.stdout.strip():
            return json.loads(out.stdout)
    except subprocess.TimeoutExpired:
        print(f"⏳ Timeout on {file_path}")
    except Exception as e:
        print(f"⚠️ Error on {file_path}: {e}")
    return None


# Use batch size = 10
batch_size = 10
results = []

for i in tqdm(range(0, len(pending_files), batch_size), desc="Batches"):
    batch = pending_files[i:i+batch_size]

    for file_path in batch:
        result = run_checkov(file_path, timeout=30)
        if result and "results" in result:
            for failed_check in result["results"].get("failed_checks", []):
                results.append({
                    "file": file_path,
                    "check_id": failed_check.get("check_id"),
                    "check_name": failed_check.get("check_name"),
                    "severity": failed_check.get("severity"),
                    "resource": failed_check.get("resource"),
                    "guideline": failed_check.get("guideline")
                })

    # Save after each batch (resume safe)
    df = pd.DataFrame(results)
    if not df.empty:
        if os.path.exists("checkov_results_partial.csv"):
            df.to_csv("checkov_results_partial.csv", mode="a", header=False, index=False)
        else:
            df.to_csv("checkov_results_partial.csv", index=False)

    results = []  # free memory
    break  # <-- only run 1 batch for testing


Total IaC files: 35966
Pending files: 35966


Batches:   0%|          | 0/3597 [00:48<?, ?it/s]
